<a href="https://colab.research.google.com/github/Wanita-8943/efficientnet_keras_transfer_learning/blob/master/%E0%B8%87%E0%B8%B2%E0%B8%99%E0%B8%9F%E0%B8%B1%E0%B8%99%E0%B8%82%E0%B8%AD%E0%B8%87%E0%B9%80%E0%B8%A3%E0%B8%B2_Keras_efficientnet_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [How to do Transfer learning with Efficientnet](https://www.dlology.com/blog/transfer-learning-with-efficientnet/) | DLology

## Imports

In [45]:
!sudo update-alternatives --config python3 #เปลี่ยน version ของ python 

There are 2 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                Priority   Status
------------------------------------------------------------
  0            /usr/bin/python3.7   2         auto mode
* 1            /usr/bin/python3.6   1         manual mode
  2            /usr/bin/python3.7   2         manual mode

Press <enter> to keep the current choice[*], or type selection number: 1


In [46]:
!apt-get install python3-pip #ติดตั้ง pip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-pip is already the newest version (9.0.1-2.3~ubuntu1.18.04.5).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [47]:
!python3 -m pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
# Downgrade pillow to avoid `UserWarning: Possibly corrupt EXIF data.`
!pip install pillow==4.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import os
import glob
import shutil
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline


## Hyper parameters

In [50]:
#กำหนดค่าพารามิเตอร์
batch_size = 5

width = 150
height = 150
epochs = 20
NUM_TRAIN = 2000
NUM_TEST = 1000
dropout_rate = 0.2
input_shape = (height, width, 3)

## Clone **efficientnet** repo

In [51]:
#ดึงข้อมูลใน Github มาใช้
import os
%cd /content
if not os.path.isdir("efficientnet_keras_transfer_learning"):
 !git clone https://github.com/Wanita-8943/efficientnet_keras_transfer_learning
%cd efficientnet_keras_transfer_learning/

/content
/content/efficientnet_keras_transfer_learning


## Import efficientnet and load the conv base model

In [52]:
# Options: EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3
# Higher the number, the more complex the model is.
from efficientnet import EfficientNetB0 as Net
from efficientnet import center_crop_and_resize, preprocess_input

In [53]:
#!pip install tf-nightly

In [54]:
# loading pretrained conv base model
# โหลดโมเดล มาโดยตัด output ของโมเดลออก เเต่ยังใช้ input อันเดิม
# เเละโหลด weight ของโมเดล มาด้วยที่ชื่อว่า imagenet
conv_base = Net(weights='imagenet', include_top=False, input_shape=input_shape)

In [55]:
conv_base.summary() #ดู Summary

Model: "efficientnet-b0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_130 (Conv2D)            (None, 75, 75, 32)   864         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_98 (BatchN  (None, 75, 75, 32)  128         ['conv2d_130[0][0]']             
 ormalization)                                                                                    
                                                                                    

## Download and extract `dog_vs_cat` dataset

In [56]:
#https://drive.google.com/drive/folders/1he-7wku7JhK5qUKw90h46FHSZTDJSu8X?usp=sharing

In [57]:
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/My Drive/Panoramic'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [58]:
path = '/content/drive/My Drive/Panoramic'

In [59]:
#if not os.path.isfile("kagglecatsanddogs_5340.zip"):
 #!wget /content/kagglecatsanddogs_5340.zip
 #!unzip -qq /content/kagglecatsanddogs_5340.zip -d dog_vs_cat
#else:
 # print("dataset already exist.")

In [60]:
data_file_path = os.path.join(path,'/content/drive/My Drive/Panoramic/Pilot_film')   

In [61]:
#!ls /content/drive/My Drive/dog_vs_cat/data
#!ls /content/drive/My Drive/dog_vs_cat/data/cat -l -t | head -10

In [64]:
#glob.glob คือ เพื่อค้นหาไฟล์ที่ต้องการใน python
# The path to the directory where the original #len นับจำนวน
# dataset was uncompressed #.format จัดรูปแบบข้อความ
original_dataset_dir = '/content/drive/My Drive/Panoramic/Pilot_film'

EightYF_images = glob.glob(os.path.join(original_dataset_dir, "8years_F", '*.jpg'))
NineYF_images = glob.glob(os.path.join(original_dataset_dir, "9years_F", '*.jpg'))
TenYF_images = glob.glob(os.path.join(original_dataset_dir, "10years_F", '*.jpg'))
print("total 8years_F images: {}\n\rtotal 9years_F images: {}\n\rtotal 10years_F images: {}".format(len(EightYF_images), len(NineYF_images), len(TenYF_images)))

total 8years_F images: 11
total 9years_F images: 11
total 10years_F images: 10


In [65]:
# ไดเรกทอรีที่เราจะจัดเก็บข้อมูลที่มีขนาดเล็กลง
#Directory คือสารบัญที่เก็บรวบรวมรายชื่อไฟล์ต่างๆ
base_dir = '/content/drive/My Drive/Panoramic/Pilot_film'
os.makedirs(base_dir, exist_ok=True) #สร้างโฟลเดอร์ จาก base_dir

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir, exist_ok=True)
validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir, exist_ok=True)
test_dir = os.path.join(base_dir, 'test')
os.makedirs(test_dir, exist_ok=True)

# Directory with our training cat pictures
train_8years_F_dir = os.path.join(train_dir, '8years_F')
os.makedirs(train_8years_F_dir, exist_ok=True)

# Directory with our training dog pictures
train_9years_F_dir = os.path.join(train_dir, '9years_F')
os.makedirs(train_9years_F_dir, exist_ok=True)

train_10years_F_dir = os.path.join(train_dir, '10years_F')
os.makedirs(train_10years_F_dir, exist_ok=True)

# Directory with our validation cat pictures
validation_8years_F_dir = os.path.join(validation_dir, '8years_F')
os.makedirs(validation_8years_F_dir, exist_ok=True)

# Directory with our validation dog pictures
validation_9years_F_dir = os.path.join(validation_dir, '9years_F')
os.makedirs(validation_9years_F_dir, exist_ok=True)

validation_10years_F_dir = os.path.join(validation_dir, '10years_F')
os.makedirs(validation_10years_F_dir, exist_ok=True)

# Directory with our validation cat pictures
test_8years_F_dir = os.path.join(test_dir, '8years_F')
os.makedirs(test_8years_F_dir, exist_ok=True)

# Directory with our validation dog pictures
test_9years_F_dir = os.path.join(test_dir, '9years_F')
os.makedirs(test_9years_F_dir, exist_ok=True)

test_10years_F_dir = os.path.join(test_dir, '10years_F')
os.makedirs(test_10years_F_dir, exist_ok=True)

# Copy first NUM_TRAIN//2 cat images to train_cats_dir # // หารปัดเศษทิ้ง	
fnames = EightYF_images[:NUM_TRAIN//2]
for fname in fnames:
    dst = os.path.join(train_8years_F_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
offset = NUM_TRAIN//2
# Copy next NUM_TEST //2 cat images to validation_cats_dir
fnames = EightYF_images[offset:offset + NUM_TEST // 2]
for fname in fnames:
    dst = os.path.join(validation_8years_F_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
offset = offset + NUM_TEST // 2
# Copy next NUM_TRAIN//2 cat images to test_cats_dir
fnames = EightYF_images[offset:offset + NUM_TEST // 2]
for fname in fnames:
    dst = os.path.join(EightYF_images, os.path.basename(fname))
    shutil.copyfile(fname, dst)

# Copy first NUM_TRAIN//2 cat images to train_cats_dir # // หารปัดเศษทิ้ง	
fnames = NineYF_images[:NUM_TRAIN//2]
for fname in fnames:
    dst = os.path.join(train_9years_F_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
offset = NUM_TRAIN//2
# Copy next NUM_TEST //2 cat images to validation_cats_dir
fnames = NineYF_images[offset:offset + NUM_TEST // 2]
for fname in fnames:
    dst = os.path.join(validation_9years_F_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
offset = offset + NUM_TEST // 2
# Copy next NUM_TRAIN//2 cat images to test_cats_dir
fnames = NineYF_images[offset:offset + NUM_TEST // 2]
for fname in fnames:
    dst = os.path.join(NineYF_images, os.path.basename(fname))
    shutil.copyfile(fname, dst)

    # Copy first NUM_TRAIN//2 cat images to train_cats_dir # // หารปัดเศษทิ้ง	
fnames = TenYF_images[:NUM_TRAIN//2]
for fname in fnames:
    dst = os.path.join(train_10years_F_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
offset = NUM_TRAIN//2
# Copy next NUM_TEST //2 cat images to validation_cats_dir
fnames = TenYF_images[offset:offset + NUM_TEST // 2]
for fname in fnames:
    dst = os.path.join(validation_10years_F_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
offset = offset + NUM_TEST // 2
# Copy next NUM_TRAIN//2 cat images to test_cats_dir
fnames = TenYF_images[offset:offset + NUM_TEST // 2]
for fname in fnames:
    dst = os.path.join(EightYF_images, os.path.basename(fname))
    shutil.copyfile(fname, dst)

In [66]:
#os.listdir แสดงลิสต์ของชื่อไฟล์ทั้งหมดในโฟลเดอร์ที่ระบุ
#len นับจำนวน
print('total training 8years_F images:', len(os.listdir(train_8years_F_dir))) 
print('total training 9years_F images:', len(os.listdir(train_9years_F_dir)))
print('total training 10years_F images:', len(os.listdir(train_10years_F_dir)))
print('total validation 8years_F images:', len(os.listdir(validation_8years_F_dir)))
print('total validation 9years_F images:', len(os.listdir(validation_9years_F_dir)))
print('total validation 10years_F images:', len(os.listdir(validation_10years_F_dir)))
print('total test 8years_F images:', len(os.listdir(test_8years_F_dir)))
print('total test 9years_F images:', len(os.listdir(test_9years_F_dir)))
print('total test 10years_F images:', len(os.listdir(test_10years_F_dir)))

total training 8years_F images: 11
total training 9years_F images: 11
total training 10years_F images: 10
total validation 8years_F images: 6
total validation 9years_F images: 6
total validation 10years_F images: 5
total test 8years_F images: 5
total test 9years_F images: 6
total test 10years_F images: 6


In [67]:
# Train ด้วย ImageDataGenerator ของ Keras ซึ่งจะเพิ่มข้อมูลเสริมระหว่างการฝึกเพื่อลดโอกาสเกิด overfitting
#overfitting เกิดจากข้อมูลที่ซับซ้อนกันเกินไป
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255, #โมเดลส่วนใหญ่ต้องใช้ RGB ในช่วง 0–1
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory #ไดเรกเป้าหมาย
        train_dir,
        # รูปภาพทั้งหมดจะถูกปรับขนาดตามความสูงและความกว้างของเป้าหมาย
        target_size=(height, width),
        batch_size=batch_size,
        # Since we use categorical_crossentropy loss, we need categorical labels
        #เนื่องจากเราใช้ categorical_crossentropy loss เราจึงต้องมีป้ายกำกับตามหมวดหมู่
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory( #การดึงภาพจาก Directory มาเข้าโมเดล 
        validation_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

Found 32 images belonging to 3 classes.
Found 17 images belonging to 3 classes.


#ยังไม่เข้าใจ

In [68]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalMaxPooling2D(name="gap"))
# model.add(layers.Flatten(name="flatten"))
if dropout_rate > 0:
    model.add(layers.Dropout(dropout_rate, name="dropout_out"))
# model.add(layers.Dense(256, activation='relu', name="fc1"))
model.add(layers.Dense(2, activation='softmax', name="fc_out"))

In [69]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnet-b0 (Functional  (None, 5, 5, 1280)       4049564   
 )                                                               
                                                                 
 gap (GlobalMaxPooling2D)    (None, 1280)              0         
                                                                 
 dropout_out (Dropout)       (None, 1280)              0         
                                                                 
 fc_out (Dense)              (None, 2)                 2562      
                                                                 
Total params: 4,052,126
Trainable params: 4,010,110
Non-trainable params: 42,016
_________________________________________________________________


In [70]:
print('This is the number of trainable layers '
      'before freezing the conv base:', len(model.trainable_weights))

conv_base.trainable = False

print('This is the number of trainable layers '
      'after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable layers before freezing the conv base: 213
This is the number of trainable layers after freezing the conv base: 2


In [72]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])
history = model.fit_generator(
      train_generator,
      steps_per_epoch= NUM_TRAIN //batch_size,
      epochs=epochs,
      validation_data=validation_generator,
      validation_steps= NUM_TEST //batch_size,
      verbose=1,
      use_multiprocessing=True,
      workers=4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if sys.path[0] == '':


Epoch 1/20


InvalidArgumentError: ignored

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_x = range(len(acc))

plt.plot(epochs_x, acc, 'bo', label='Training acc')
plt.plot(epochs_x, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs_x, loss, 'bo', label='Training loss')
plt.plot(epochs_x, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Fine tuning last several layers.

In [ ]:
# multiply_16
# set 'multiply_16' and following layers trainable
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'multiply_16':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch= NUM_TRAIN //batch_size,
      epochs=epochs,
      validation_data=validation_generator,
      validation_steps= NUM_TEST //batch_size,
      verbose=1,
      use_multiprocessing=True,
      workers=4)

In [ ]:
os.makedirs("./models", exist_ok=True)
model.save('./models/cats_and_dogs_small.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_x = range(len(acc))

plt.plot(epochs_x, acc, 'bo', label='Training acc')
plt.plot(epochs_x, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs_x, loss, 'bo', label='Training loss')
plt.plot(epochs_x, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Test predict

In [ ]:
cat_img = cat_images[-1]
dog_img= dog_images[-1]

In [ ]:
Image(filename=cat_img)

In [ ]:
Image(filename=dog_img)

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model.predict([x])[0][0]
    if result > 0.5:
        animal = "cat"
    else:
        animal = "dog"
        result = 1 - result
    return animal,result

print(predict_image(cat_img))
print(predict_image(dog_img))

## Download the model

In [ ]:
from google.colab import files

files.download('./models/cats_and_dogs_small.h5')

In [ ]:
model.input_shape

## Load the model

In [ ]:
from efficientnet.layers import Swish, DropConnect
from efficientnet.model import ConvKernalInitializer
from tensorflow.keras.utils import get_custom_objects

get_custom_objects().update({
    'ConvKernalInitializer': ConvKernalInitializer,
    'Swish': Swish,
    'DropConnect':DropConnect
})

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("./models/cats_and_dogs_small.h5")